# 🔗 The NeRF Geometry Lab
### Interactive Exploration of the "Hidden Math" Behind Protein AI

Most modern protein AI models (like AlphaFold and trRosetta) don't predict 3D coordinates directly. Instead, they predict **internal coordinates** (Bond Lengths, Angles, and Torsions) and then use an algorithm called **NeRF** (Natural Extension Reference Frame) to build the 3D model atom-by-atom.

In this tutorial, we will:
1. Explore the **Z-Matrix** (Internal Coordinate representation).
2. Use interactive sliders to change Backbone Torsions ($\phi$ and $\psi$).
3. Observe how "Local" rotational changes cause "Global" downstream swings.

In [ ]:
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from ipywidgets import interact, IntSlider, FloatSlider
import os, sys, numpy as np
import py3Dmol
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

# --- UNIVERSAL SETUP ---
pio.renderers.default = 'vscode'
sys.path.append(os.path.abspath('../../'))
from synth_pdb import PeptideGenerator, EnergyMinimizer, PDBValidator, PeptideResult
import biotite.structure as struc

# Robust 3Dmol.js injection
display(HTML('<script src="https://3Dmol.org/build/3Dmol-min.js"></script>'))
print("[✅] Environment Ready. Using internal synth-pdb engine for geometry.")

## 1. Setting the Scene
We'll start with a simple Poly-Alanine peptide (10 residues) in an alpha-helical conformation.

In [ ]:
sequence = "ALA-ALA-ALA-ALA-ALA-ALA-ALA-ALA-ALA-ALA"
gen = PeptideGenerator(sequence)
peptide = gen.generate(conformation="alpha")
baseline_struct = peptide.structure.copy()

print("Baseline alpha-helix generated.")

## 2. Exploring the Z-Matrix
A **Z-Matrix** is a way of describing the position of each atom relative to three previously defined atoms. ItTypically consists of:
- **Bond Length**: Distance to the parent atom.
- **Bond Angle**: Angle between the current atom, its parent, and its grandparent.
- **Dihedral Angle**: Torsion between the current atom and three ancestors.

In protein AI, we focus on the backbone dihedrals. Use the slider below to inspect the internal coordinates of various residues.

In [ ]:
def show_z_matrix(res_idx):
    # Ideal Alpha Helix angles
    phi, psi, omega = -57.0, -47.0, 180.0
    
    df = pd.DataFrame({
        'Parameter': ['Phi (φ)', 'Psi (ψ)', 'Omega (ω)'],
        'Angle (Degrees)': [phi, psi, omega]
    })
    
    print(f"--- Ideal internal coordinates for Residue {res_idx + 1} ---")
    return df

interact(show_z_matrix, res_idx=IntSlider(min=0, max=9, step=1, value=4, description='Residue:'));

## 3. Interactive Geometry Lab
Rotate the Phi angle of the 5th residue and watch the effect on the rest of the chain.

In [ ]:
def lab_view(phi_offset, psi_offset):
    view = py3Dmol.view(width=600, height=400)
    
    res_idx = 4 # 5th residue
    n_res = len(sequence.split('-'))
    
    # Define base angles (alpha helix)
    phis = [-57.0] * n_res
    psis = [-47.0] * n_res
    
    # Apply modifications
    phis[res_idx] += phi_offset
    psis[res_idx] += psi_offset
    
    # Regenerate structure atom-by-atom using synth-pdb internal coordinate engine
    # This is exactly what NeRF does!
    res = gen.generate(phi_list=phis, psi_list=psis)
    
    view.addModel(res.pdb, "pdb")
    view.setStyle({'stick': {'color': 'spectrum'}})
    
    # Highlight the modified residue
    view.setStyle({'resv': [res_idx+1]}, {'stick': {'color': 'red'}})
    view.zoomTo()
    return view.show()

interact(lab_view, 
         phi_offset=FloatSlider(min=-180, max=180, step=10, value=0, description='dPhi'),
         psi_offset=FloatSlider(min=-180, max=180, step=10, value=0, description='dPsi'));

### 🧠 The Mathematical Insight
Notice how small changes in $\phi$ or $\psi$ early in the sequence cause **massive displacements** at the C-terminus. This is precisely why it is difficult for AI models to predict coordinates directly - a 1-degree error in a dihedral angle can result in a 20Å error at the end of the protein. 

This is why models like **AlphaFold2** use the **Invariant Point Attention (IPA)** and Frame-based updates to correct these compounding errors!